In [15]:
from functools import partial
import scipy as sp
import numpy as np
from skbio import DNA ,TabularMSA ,TreeNode ,DistanceMatrix ,Sequence,Protein
from skbio.alignment import global_pairwise_align



def kmer_distance(sequence1, sequence2, k=7, overlap=False):
    sequence1_kmers = set(map(str, sequence1.iter_kmers(k, overlap)))
    sequence2_kmers = set(map(str, sequence2.iter_kmers(k, overlap)))
    all_kmers = sequence1_kmers | sequence2_kmers
    shared_kmers = sequence1_kmers & sequence2_kmers
    number_unique = len(all_kmers) - len(shared_kmers)
    fraction_unique = number_unique / len(all_kmers)
    return fraction_unique

def hamming_distance(sequence1, sequence2):
    n,m = len(sequence1),len(sequence2)
    return sum(sequence1[i] != sequence2[i] for i in range(min(n,m))) + abs(n-m)

def minimum_edit_distance(sequence1,sequence2):
     # dynamic programming
    n,m = len(sequence1),len(sequence2)
    dp = np.zeros((n+1,m+1))
    for i in range(n+1):
        dp[i][0] = i
    for j in range(m+1):
        dp[0][j] = j
    for i in range(1,n+1):
        for j in range(1,m+1):
            dp[i][j] = min(dp[i-1][j]+1,dp[i][j-1]+1,dp[i-1][j-1]+(sequence1[i-1] != sequence2[j-1]))
    return dp[n][m]

def alignement_score_distance(sequence1,sequence2):
    if sequence1 == sequence2:
        return 0
    _,score, _ = global_pairwise_align_nucleotide(sequence1, sequence2)
    return score


def guide_tree_from_sequences(sequences, metric=hamming_distance):
        guide_dm = DistanceMatrix.from_iterable(sequences, metric=metric, key='id')
        print(guide_dm)
        guide_lm = sp.cluster.hierarchy.average(guide_dm.condensed_form())
        guide_tree = TreeNode.from_linkage_matrix(guide_lm, guide_dm.ids)
        return guide_tree
def progressive_msa(sequences, pairwise_aligner, guide_tree=None,metric=kmer_distance):

    if guide_tree is None:
        guide_dm = DistanceMatrix.from_iterable(sequences, metric=metric, key='id')
        guide_lm = sp.cluster.hierarchy.average(guide_dm.condensed_form())
        guide_tree = TreeNode.from_linkage_matrix(guide_lm, guide_dm.ids)

    seq_lookup = {s.metadata['id']: s for i, s in enumerate(sequences)}
    c1, c2 = guide_tree.children
    c1_aln = seq_lookup[c1.name] if c1.is_tip() else progressive_msa(sequences, pairwise_aligner, c1)
    c2_aln = seq_lookup[c2.name] if c2.is_tip() else progressive_msa(sequences, pairwise_aligner, c2)

    alignment, _, _ = pairwise_aligner(c1_aln, c2_aln)

    return alignment


In [2]:
query_of_sequences = [DNA("ACGCGATGACCGGGCCTTGTAAAAT", metadata={'id': 'seq1'}),
                        DNA("ATGATGACAGGGCTTGTAACT", metadata={'id': 'seq2'}),
                        DNA("TTCATGACCGGCTTATACTTAT", metadata={'id': 'seq3'}),
                        DNA("ACCCTACCTGTCGTATTGTAAT", metadata={'id': 'seq4'}),
                      ]
guide_tree = guide_tree_from_sequences(query_of_sequences,metric=kmer_distance)
guide_tree = guide_tree_from_sequences(query_of_sequences,metric=hamming_distance)
guide_tree = guide_tree_from_sequences(query_of_sequences,metric=minimum_edit_distance)
guide_tree = guide_tree_from_sequences(query_of_sequences,metric=alignement_score_distance)

4x4 distance matrix
IDs:
'seq1', 'seq2', 'seq3', 'seq4'
Data:
[[0. 1. 1. 1.]
 [1. 0. 1. 1.]
 [1. 1. 0. 1.]
 [1. 1. 1. 0.]]
4x4 distance matrix
IDs:
'seq1', 'seq2', 'seq3', 'seq4'
Data:
[[ 0. 25. 25. 25.]
 [25.  0. 22. 22.]
 [25. 22.  0. 22.]
 [25. 22. 22.  0.]]
4x4 distance matrix
IDs:
'seq1', 'seq2', 'seq3', 'seq4'
Data:
[[ 0. 25. 25. 25.]
 [25.  0. 22. 22.]
 [25. 22.  0. 22.]
 [25. 22. 22.  0.]]


/home/codespace/.python/current/lib/python3.10/site-packages/skbio/alignment/_pairwise.py:595: EfficiencyWarning: You're using skbio's python implementation of Needleman-Wunsch alignment. This is known to be very slow (e.g., thousands of times slower than a native C implementation). We'll be adding a faster version soon (see https://github.com/biocore/scikit-bio/issues/254 to track progress on this).
  warn("You're using skbio's python implementation of Needleman-Wunsch "


4x4 distance matrix
IDs:
'seq1', 'seq2', 'seq3', 'seq4'
Data:
[[0. 2. 1. 0.]
 [2. 0. 2. 2.]
 [1. 2. 0. 1.]
 [0. 2. 1. 0.]]


In [3]:
aligner = partial(global_pairwise_align_nucleotide,match_score = 6,mismatch_score = -2, gap_open_penalty=4, gap_extend_penalty=1)
MSA = progressive_msa(query_of_sequences, pairwise_aligner=aligner,guide_tree=guide_tree)
print(MSA)

TabularMSA[DNA]
----------------------------
Stats:
    sequence count: 4
    position count: 28
----------------------------
AT--GATGACAGGGC---TTGTAACT--
-TTC-ATGACC-GGC---TTATACTTAT
ACGCGATGACCGGGC--CTTGTAAA-AT
ACCC--T-ACCTGTCGTATTGTAA---T


In [16]:
from pathlib import Path
import skbio.io
from tqdm import tqdm
fasta_files = list(Path("./Oxbench_input").glob("*.fa"))


# read all fasta files in the directory and align them using progressive_align.py then save the alignment in Oxbench_output_naive_approach
# make it output the alignment in fasta format
for fasta_file in tqdm(fasta_files):
    sequences = []
    for seq in skbio.io.read(str(fasta_file), format="fasta"):
        sequences.append(Protein(seq))
    guide_tree = guide_tree_from_sequences(sequences,metric=alignement_score_distance)
    aligner = partial(global_pairwise_align,match_score = 6,mismatch_score = -2, gap_open_penalty=4, gap_extend_penalty=1)
    MSA = progressive_msa(sequences, pairwise_aligner=aligner,guide_tree=guide_tree)
    skbio.io.write(MSA, str(fasta_file).replace("Oxbench_input","Oxbench_output_naive_approach"), format="fasta")
        

  0%|          | 0/10 [00:00<?, ?it/s]


TypeError: `seq1` and `seq2` must be DNA, RNA, or TabularMSA, not type 'Protein'